In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sp

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import kruskal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']

skf = StratifiedKFold(n_splits=10) 

test = False
test2 = False
test3 = False
test4 = False

for train_index, test_index in skf.split(X, y):
   

    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    # utilisation de la méthode gaussienne 
    gmm = GaussianMixture(n_components=2,init_params='kmeans',covariance_type='diag') # appel  de la fonction gaussian mixture
    gmm.fit(X_train,y_train) # entrainement du modèle 
    y_pred_gmm=gmm.predict(X_test)# prédiction du modèle sur les X_test
    if precision_score(y_test,y_pred_gmm)< 0.05: # interversion des labels en cas de problème de labelisation

        y_pred_gmm = np.array([0 if label == 1 else 1 for label in y_pred_gmm])
    
    F1=round(f1_score(y_test,y_pred_gmm),3) # récupération de la valeur F1_score


    if test==False :
            data = np.array([[F1]]) # on rentre les valeurs de F1 dans un array
            test=True
    else :
            data = np.append(data,np.array([F1]).reshape(1,1),axis=0) # si l'array est déja crée, on ajoute les scores dans le array déja crée
    ########
    # utilisation de la méthode KNN
    model_KNN = KNeighborsClassifier(n_neighbors=16, weights='distance', algorithm='auto', metric='braycurtis')# appel de la méthode KNN
    model_KNN.fit(X_train,y_train)# entrainement du modèle
    y_pred_KNN = model_KNN.predict(X_test)# prédiction du modèle sur X_test
    F1_knn=round(f1_score(y_test,y_pred_KNN),3)# récupération du score F1 pour KNN
    
    # même logique ici que pour gaussian mixture
    if test2==False :
            KNN = np.array([[F1_knn]])
            test2=True
    else :
            KNN = np.append(KNN,np.array([F1_knn]).reshape(1,1),axis=0)
    ###########
    # utilisation du modèle réseaux de neurones 
    model_PMC_1 = MLPClassifier(solver='lbfgs',alpha=0.1, hidden_layer_sizes=(4, 4),random_state=1 )# appel de la méthode réseaux de neurones
    model_PMC_1.fit(X_train, y_train)# entrainement du modèle
    y_pred_PMC_1= model_PMC_1.predict(X_test)# prédiction du modèle sur X_test
    if precision_score(y_test,y_pred_PMC_1)< 0.05: # interversion des labels en cas de problème de labelisation

        y_pred_PMC_1 = np.array([0 if label == 1 else 1 for label in y_pred_PMC_1])
    F1_Neurones=round(f1_score(y_test, y_pred_PMC_1),3)# récupération du score F1
    if test3==False :
            Neurones = np.array([[F1_Neurones]])
            test3=True
    else :
            Neurones = np.append(Neurones,np.array([F1_Neurones]).reshape(1,1),axis=0)
   #############
   # utilisation du modèle  perceptron 
    PerceP = Perceptron(penalty ='l1',alpha =1,l1_ratio=1e-6, max_iter=50) # appel de la méthode perceptron 
    PerceP.fit(X_train, y_train)# entrainement du modèle
    y_pred_PSC = PerceP.predict(X_test)# prédiction du modèle 
    F1_Perceptron=round(f1_score(y_test, y_pred_PSC),3) # récupération du score F1
    if test4==False :
            perceptron = np.array([[F1_Perceptron]])
            test4=True
    else :
            perceptron = np.append(perceptron,np.array([F1_Perceptron]).reshape(1,1),axis=0)
   ##########
# conversion des np.array en dataframe  
df = pd.DataFrame(data,columns=['[{}]'.format("Gaussian")])
df1 = pd.DataFrame(KNN,columns=['[{}]'.format("KNN")])
df2 = pd.DataFrame(Neurones,columns=['[{}]'.format("Réseaux neurones")])
df3 = pd.DataFrame(perceptron,columns=['[{}]'.format("Perceptron")])

Comparaison = pd.concat([df, df1,df2,df3], axis = 1) # on concatène toutes les colonnes

display(Comparaison)

In [ ]:
import scikit_posthocs as sp

# comparaison des variances avec le t-statistiques et la p-value
t,p = kruskal(Comparaison['[Gaussian]'],Comparaison['[KNN]'],Comparaison['[Réseaux neurones]'],Comparaison['[Perceptron]'])
print('t statistic: %.3f' % t)
print('p value: %.20f' % p)
# comparaison des variances entre algorithme 1 à 1
sp.posthoc_conover([Comparaison['[Gaussian]'],Comparaison['[KNN]'],Comparaison['[Réseaux neurones]'],Comparaison['[Perceptron]']])